In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import cv2
# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%config Completer.use_jedi = False


In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
data = pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv')

In [ ]:
data.head()

In [ ]:
img = cv2.imread('/kaggle/input/state-farm-distracted-driver-detection/imgs/train/c0/img_25094.jpg')


In [ ]:
img.shape

In [ ]:
def get_im_cv2(path, img_rows, img_cols, color_type=3):
    # Load as grayscale
#     if color_type == 1:
    img = cv2.imread(path,0)
#     elif color_type == 3:
#         img = cv2.imread(path)
    #Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

In [ ]:
X = data[['img']]
y = data[['classname']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train.columns,y_train.columns)

In [ ]:

plt.figure(figsize=(30,30))
for i in range(5):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(True)
    path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train/{}/{}".format(y.loc[i+3,'classname'],X.loc[i+3,'img'])
    train_image= get_im_cv2(path,200,200)
    print(train_image.shape)
#     print(train_image)
#     print(type(train_image))

    plt.imshow(train_image)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel("_{}".format(train_image.shape))
plt.show()


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [ ]:
 model.summary()

    

In [ ]:
image_train_arr = []
for i in range(len(X_train)):
#     print((X_train.loc[i,'']))
#     print(y_train.iloc[i,0],X_train.iloc[i,0])
#     break

    path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train/{}/{}".format(y_train.iloc[i,0],X_train.iloc[i,0])
    resized = get_im_cv2(path,32,32)
    image_train_arr.append(resized)
#     print(data.loc[i,'images'])
    

In [ ]:
image_test_arr = []
for i in range(len(X_test)):
#     print((X_train.loc[i,'']))
#     print(y_train.iloc[i,0],X_train.iloc[i,0])
#     break

    path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train/{}/{}".format(y_test.iloc[i,0],X_test.iloc[i,0])
    resized = get_im_cv2(path,32,32)
    image_test_arr.append(resized)
#     print(data.loc[i,'images'])

In [ ]:
image_train_arr = np.array(image_train_arr)
image_test_arr = np.array(image_test_arr)
print(image_train_arr.shape,image_test_arr.shape)


In [ ]:
image_train_arr,image_test_arr = image_train_arr/255.0, image_test_arr/255.0

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()


In [ ]:
type(image_test_arr),type(y_test)


In [ ]:
image_test_arr.shape,y_test.shape

In [ ]:
image_train_arr = image_train_arr.reshape(15024,32,32,1)
image_test_arr = image_test_arr.reshape(7400,32,32,1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_train['new-col'] = labelencoder.fit_transform(y_train)
y_test['new-col'] = labelencoder.fit_transform(y_test)


In [ ]:
y_train

In [ ]:
y_test['new-col']

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(image_train_arr, y_train['new-col'], epochs=30,validation_data=(image_test_arr, y_test['new-col']))


With 10 Epoch we got 0.9723 accuracy
With 30 Epoch we got 0.9723 accuracy


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1.1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(image_test_arr, y_test['new-col'], verbose=2)

In [ ]:
test_data = pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv')
test_data.shape

In [ ]:
test_data.columns

In [ ]:
y_test

In [ ]:
test_arr = []
for i in range(len(test_data)):
#     print((X_train.loc[i,'']))
#     print(y_train.iloc[i,0],X_train.iloc[i,0])
#     break

    path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test/{}".format(test_data.iloc[i,0])
    resized = get_im_cv2(path,32,32)
    test_arr.append(resized)
#     print(data.loc[i,'images'])

In [ ]:
test_arr=np.array(test_arr)


In [ ]:
test_arr.shape

In [ ]:
test_arr_norm =test_arr/255.0

In [ ]:
test_arr_norm= test_arr_norm.reshape(79726,32,32,1)

In [ ]:
test_output_arr = model.predict(test_arr_norm)

In [ ]:
test_output_arr

In [ ]:
test_output_arr.shape

In [ ]:
test_output_prob = pd.DataFrame(test_output_arr)


In [ ]:
type(test_output_prob)

In [ ]:
for i in range (len(test_output_prob.columns)):
    test_output_prob["c{}".format(i)] = test_output_prob.iloc[:,i]
print(test_output_prob.columns)

In [ ]:
test_output_prob.shape

In [ ]:
test=test_output_prob.drop([0,1,2,3,4,5,6,7,8,9], axis = 1) 


In [ ]:
test.columns


In [ ]:
result = pd.concat([test_data.iloc[:,0], test], axis=1)

In [ ]:
result.shape,result.columns

In [ ]:
result.isnull().values.any()

In [ ]:
result.head()

In [ ]:
result.to_csv(r'result.csv',index=False)